# Finding regions for each LSOA

Match LSOAs to the regions that contain them.

The final file will contain this information:

| Column name | Description | Usage |
| --- | --- | --- | 
| LSOA11 CD / NM | LSOA 2011 codes / names | England & Wales |
| long / lat | Longitude and latitude of the centroid of the LSOA | England & Wales |
| CCG19 CD / NM | Clinical Commissioning Groups 2019 codes / names | England |
| ICB22 CD / NM | Integrated Care Board 2022 codes / names. Replacement for CCGs. | England |
| STP19 CD / NM | Sustainability and Transformation Partnerships codes / names (~similar to counties) | England |
| LHB20 CD / NM / NMW | Local Health Boards 2020 codes / names / Welsh names | Wales |
| RGN11 CD / NM | Region codes / names. Gives specific regions within England, but Wales and Scotland just get the country name. | England & Wales |
| LAD17 CD / NM | Local Authority District 2017 codes / names | England & Wales |
| SCN17 CD/NM | Strategic Clinical Network 2017 codes / names | England |

To match the LSOAs to regions, there are a number of files from the Office for National Statistics that allow cross-matching.

| File | Provides |
| --- | --- |
| LSOA (2011) to Clinical Commissioning Groups to Sustainability and Transformation Partnerships (April 2019) Lookup in England | CCG 2019 names and codes, STP 2019 names and codes |
| LSOA (2011) to Sub ICB Locations to Integrated Care Boards to Local Authority Districts (July 2022) Lookup in England | ICB 2022 names and codes |
| Output Areas (2011) to Local Health Boards (December 2020) Lookup in Wales | OA 2011 codes, LHB codes, names, and Welsh names |
| Output Area to LSOA to MSOA to Local Authority District (December 2017) Lookup with Area Classifications in Great Britain | OA 2011 codes, LSOA 2011 codes and names, region codes and names, LAD codes and names |
| Local Authority District to Strategic Clinical Network (December 2017) Lookup in England | SCN 2017 codes, names |
| (geojson) LSOA (Dec 2011) Boundaries Super Generalised Clipped (BSC) EW V3 | LSOA centroids in longitude / latitude |

Two files are used for matching the Welsh LSOAs with region names because one file links LSOA to OA and the other links OA to region names.

Two files are used for matching the LSOAs with SCNs because one file links LSOA to LAD and the other links LAD to SCN.

| Source | Region          | OA    | LSOA  | MSOA  | SICBL | ICB   | CCG   | STP   | LHB   | LAD   | SCN   | Ambulance | ISDN   |
| ---    | ---             | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---:     |  :---: |
| ONS    | Great Britain   |  🟡   |   🟡   |  🟡   |       |       |       |       |       |   🟡   |      |           |        |
| ONS    | England         |       |  🔴   |       |   🔴   |  🔴   |       |       |       |       |       |           |        |
| ONS    | England         |       |  🔴   |       |       |       |   🔴   |   🔴  |       |       |       |           |        |
| ONS    | England         |       |       |       |       |       |       |       |       |   🔴   |   🔴  |           |        |
| ONS    | Wales           |  🟢   |       |       |       |       |       |       |   🟢   |       |       |           |        |
| Mixed  | England & Wales |       |   🟡   |       |       |       |       |       |       |       |       |    🟡      |        |
| Mixed  | England         |       |       |       |  🔴    |  🔴   |       |       |       |       |       |           |    🔴   |

In this notebook we will combine all of these lookup tables into the following separate files:

| File                  | OA    | LSOA  | MSOA  | SICBL | ICB   | CCG   | STP   | LHB   | LAD   | SCN   | Ambulance | ISDN   |
| ---                   | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---:     |  :---: |
| `regions_full_ew.csv` |       |   🟡  |       |  🟡     |   🟡   |  🟡   |  🟡  |   🟡  |   🟡  |  🟡    |     🟡    |   🟡   |
| `regions_lsoa_ew.csv` |       |   🟡  |       |   🟡   |  🟡   |       |       |   🟡  |       |        |           |      |
| `regions_ew.csv`      |       |       |       |  🟡   |   🟡   |       |       |  🟡   |       |       |    🟡      |   🟡    |

Files containing LSOA are stored separately from the rest to save disk space. There are over 35,000 LSOA in England and Wales and the larger region types will have very many LSOA in each region. It saves a lot of repeated information if there is one file that looks up LSOA to one region type and a separate file that looks up that region type to all of the other region types.

## Method:

We will combine the files to get the information we need by finding which LSOA are in each larger region.

Make a collection of dataframes that all have only one column in column: `LSOA11CD`, the LSOA code from 2011. Then merge all of the dataframes together based on that shared column.

1. __Link LSOA --> LHB via OA__.
    + Link the "OA --> LHB" lookup to the "OA --> LSOA --> MSOA --> LAD" lookup.
1. __Link LSOA --> ISDN via ICB__.
    + Link the "ICB --> ISDN" lookup to the "LSOA --> SICBL --> ICB" lookup.
1. __Link LSOA --> SCN via LAD__.
    + Link the "LAD --> SCN" lookup to the "OA --> LSOA --> MSOA --> LAD" lookup.
1. __Combine the linked LSOA data__.

## Notebook setup

In [1]:
# For handling the tabular data:
import pandas as pd

# For tracking paths to files:
import os
from dataclasses import dataclass

In [2]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    dir_tabular = '../data_tabular/'
    dir_ons_tabular = '../data_tabular/ons_data/'

    oa_lsoa_msoa_lad = 'Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District_(December_2017)_Lookup_with_Area_Classifications_in_Great_Britain.csv'
    lsoa_sicbl_icb = 'LSOA11_LOC22_ICB22_LAD22_EN_LU.csv'
    lsoa_ccg_std = 'LSOA_(2011)_to_Clinical_Commissioning_Groups_to_Sustainability_and_Transformation_Partnerships_(April_2019)_Lookup_in_England.csv'
    lad_scn = 'Local_Authority_District_to_Strategic_Clinical_Network_(December_2017)_Lookup_in_England_.csv'
    oa_lhb = 'Output_Areas_(2011)_to_Local_Health_Boards_(December_2020)_Lookup_in_Wales.csv'
    lsoa_ambo = 'lsoa_ambulance.csv'
    sicbl_icb_isdn = 'SICBL_ICB_ISDN.csv'

    df_output_full = 'regions_full_ew.csv'
    df_output_with_lsoa = 'regions_lsoa_ew.csv'
    df_output_without_lsoa = 'regions_ew.csv'

paths = Paths()

## Link LSOA to LHB

Load OA --> LHB lookup:

In [3]:
df_oa_lhb = pd.read_csv(os.path.join(paths.dir_ons_tabular, paths.oa_lhb))

df_oa_lhb.head(3)

,FID,OA11CD,LHB20CD,LHB20NM,LHB20NMW
0,1,W00000052,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
1,2,W00000104,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr
2,3,W00000053,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr


Load OA --> LSOA lookup:

In [4]:
df_oa_lsoa_region = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.oa_lsoa_msoa_lad)
)

df_oa_lsoa_region.head(3)

,OA11CD,OAC11CD,OAC11NM,LSOA11CD,LSOA11NM,SOAC11CD,SOAC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
0,E00060343,7d1,Ageing Communities and Families,E01011966,Hartlepool 006B,5b,Aspiring urban households,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,1
1,E00174083,7d1,Ageing Communities and Families,E01011974,Hartlepool 005B,4b,Constrained renters,E02002487,Hartlepool 005,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,2
2,E00060349,6a4,Ageing in Suburbia,E01011965,Hartlepool 006A,8b,Ageing suburbanites,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,3


Link OA --> LSOA --> LHB:

In [5]:
# Link via OA:
df_lsoa_lhb = pd.merge(
    df_oa_lhb[['OA11CD', 'LHB20CD', 'LHB20NM', 'LHB20NMW']],
    df_oa_lsoa_region[['OA11CD', 'LSOA11CD']],
    on='OA11CD', how='left'
)
# Drop the OA:
df_lsoa_lhb = df_lsoa_lhb.drop('OA11CD', axis='columns')
# Now there are very many duplicate rows because there are many OA
# in each LSOA. Remove the repeats:
df_lsoa_lhb = df_lsoa_lhb.drop_duplicates()

In [6]:
df_lsoa_lhb.head(3)

,LHB20CD,LHB20NM,LHB20NMW,LSOA11CD
0,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr,W01000012
1,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr,W01000021
9,W11000023,Betsi Cadwaladr University Health Board,Bwrdd Iechyd Prifysgol Betsi Cadwaladr,W01000022


## Link LSOA to ISDN

Load LSOA --> ICB lookup:

In [7]:
df_lsoa_icb = pd.read_csv(os.path.join(paths.dir_ons_tabular, paths.lsoa_sicbl_icb))

df_lsoa_icb.head(3)

,LSOA11CD,LSOA11NM,LOC22CD,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,LAD22CD,LAD22NM
0,E01012367,Halton 007A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
1,E01012368,Halton 003A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
2,E01012369,Halton 005A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton


Load ICB --> ISDN lookup:

In [8]:
df_icb_isdn = pd.read_csv(os.path.join(paths.dir_tabular, paths.sicbl_icb_isdn))

df_icb_isdn.head(3)

,LOC22CD,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,NHSER22CD,NHSER22CDH,NHSER22NM,ISDN
0,E38000240,93C,NHS North Central London ICB - 93C,E54000028,QMJ,NHS North Central London Integrated Care Board,E40000003,Y56,London,London
1,E38000255,A3A8R,NHS North East London ICB - A3A8R,E54000029,QMF,NHS North East London Integrated Care Board,E40000003,Y56,London,London
2,E38000256,W2U3Z,NHS North West London ICB - W2U3Z,E54000027,QRV,NHS North West London Integrated Care Board,E40000003,Y56,London,London


Link LSOA --> ICB --> ISDN:

In [9]:
df_lsoa_isdn = pd.merge(
    df_lsoa_icb[['LSOA11CD', 'ICB22CD']],
    df_icb_isdn[['ICB22CD', 'ISDN']].drop_duplicates(),
    on='ICB22CD', how='left'
)
df_lsoa_isdn = df_lsoa_isdn.drop('ICB22CD', axis='columns')

In [10]:
df_lsoa_isdn.head(3)

,LSOA11CD,ISDN
0,E01012367,Cheshire and Merseyside
1,E01012368,Cheshire and Merseyside
2,E01012369,Cheshire and Merseyside


## Link LSOA to SCN

Load LSOA --> LAD lookup:

In [11]:
df_oa_lsoa_region = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.oa_lsoa_msoa_lad)
)

df_oa_lsoa_region.head(3)

,OA11CD,OAC11CD,OAC11NM,LSOA11CD,LSOA11NM,SOAC11CD,SOAC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
0,E00060343,7d1,Ageing Communities and Families,E01011966,Hartlepool 006B,5b,Aspiring urban households,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,1
1,E00174083,7d1,Ageing Communities and Families,E01011974,Hartlepool 005B,4b,Constrained renters,E02002487,Hartlepool 005,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,2
2,E00060349,6a4,Ageing in Suburbia,E01011965,Hartlepool 006A,8b,Ageing suburbanites,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,3


Load LAD --> SCN lookup:

In [12]:
df_lad_scn = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.lad_scn)
)

df_lad_scn.head(3)

,LAD17CD,LAD17NM,SCN17CD,SCN17NM,FID
0,E07000201,Forest Heath,E55000006,East of England,1
1,E07000202,Ipswich,E55000006,East of England,2
2,E07000203,Mid Suffolk,E55000006,East of England,3


In [13]:
df_lsoa_scn = pd.merge(
    df_oa_lsoa_region[['LSOA11CD', 'LAD17CD']].drop_duplicates(),
    df_lad_scn[['LAD17CD', 'SCN17CD', 'SCN17NM']],
    on='LAD17CD', how='right',
)
df_lsoa_scn = df_lsoa_scn.drop('LAD17CD', axis='columns')

In [14]:
df_lsoa_scn.head(3)

,LSOA11CD,SCN17CD,SCN17NM
0,E01029951,E55000006,East of England
1,E01029929,E55000006,East of England
2,E01029946,E55000006,East of England


## Combine linked data

Load LSOA --> CCG --> STP lookup:

In [15]:
df_lsoa_ccg_stp = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.lsoa_ccg_std)
)
df_lsoa_ccg_stp.head(3)

,FID,LSOA11CD,LSOA11NM,CCG19CD,CCG19CDH,CCG19NM,STP19CD,STP19NM,LAD19CD,LAD19NM
0,1,E01010650,Bradford 027B,E38000019,02R,NHS Bradford Districts CCG,E54000005,West Yorkshire and Harrogate (Health and Care ...,E08000032,Bradford
1,2,E01010651,Bradford 032B,E38000019,02R,NHS Bradford Districts CCG,E54000005,West Yorkshire and Harrogate (Health and Care ...,E08000032,Bradford
2,3,E01010652,Bradford 026A,E38000019,02R,NHS Bradford Districts CCG,E54000005,West Yorkshire and Harrogate (Health and Care ...,E08000032,Bradford


In [16]:
# Only keep these columns:
df_lsoa_ccg_stp = df_lsoa_ccg_stp[
    ['LSOA11CD', 'CCG19CD', 'CCG19NM', 'STP19CD', 'STP19NM']]

Load LSOA --> SICBL --> ICB lookup:

In [17]:
df_lsoa_icb_lad = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.lsoa_sicbl_icb)
)

df_lsoa_icb_lad.head(3)

,LSOA11CD,LSOA11NM,LOC22CD,LOC22CDH,LOC22NM,ICB22CD,ICB22CDH,ICB22NM,LAD22CD,LAD22NM
0,E01012367,Halton 007A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
1,E01012368,Halton 003A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton
2,E01012369,Halton 005A,E38000068,01F,NHS Cheshire and Merseyside ICB - 01F,E54000008,QYG,NHS Cheshire and Merseyside Integrated Care Board,E06000006,Halton


In [18]:
# Only keep these columns:
df_lsoa_icb_lad = df_lsoa_icb_lad[
    ['LSOA11CD', 'LOC22CD', 'LOC22NM', 'ICB22CD', 'ICB22NM', 'LAD22CD', 'LAD22NM']]

Load LSOA --> Ambulance Service lookup:

In [19]:
df_lsoa_ambo = pd.read_csv(
    os.path.join(paths.dir_tabular, paths.lsoa_ambo)
)

df_lsoa_ambo.head(3)

,LSOA,ambulance_service,LAD22NM
0,Adur 001A,South East Coast,Adur
1,Adur 001B,South East Coast,Adur
2,Adur 001C,South East Coast,Adur


Temporary - rename LSOA column, merge in codes:

In [20]:
df_lsoa_ambo = df_lsoa_ambo.rename(columns={'LSOA': 'LSOA11NM'})

df_lsoa_ambo = pd.merge(
    df_lsoa_ambo, df_oa_lsoa_region[['LSOA11CD', 'LSOA11NM']].drop_duplicates(),
    on='LSOA11NM', how='left'
)

In [21]:
# Only keep these columns:
df_lsoa_ambo = df_lsoa_ambo[
    ['LSOA11CD', 'ambulance_service']]

Finally load LSOA --> MSOA --> RGN --> CTRY lookup:

In [22]:
df_oa_lsoa_region = pd.read_csv(
    os.path.join(paths.dir_ons_tabular, paths.oa_lsoa_msoa_lad)
)

df_oa_lsoa_region.head(3)

,OA11CD,OAC11CD,OAC11NM,LSOA11CD,LSOA11NM,SOAC11CD,SOAC11NM,MSOA11CD,MSOA11NM,LAD17CD,LAD17NM,LACCD,LACNM,RGN11CD,RGN11NM,CTRY11CD,CTRY11NM,FID
0,E00060343,7d1,Ageing Communities and Families,E01011966,Hartlepool 006B,5b,Aspiring urban households,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,1
1,E00174083,7d1,Ageing Communities and Families,E01011974,Hartlepool 005B,4b,Constrained renters,E02002487,Hartlepool 005,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,2
2,E00060349,6a4,Ageing in Suburbia,E01011965,Hartlepool 006A,8b,Ageing suburbanites,E02002488,Hartlepool 006,E06000001,Hartlepool,6a2r,Mining Legacy,E12000001,North East,E92000001,England,3


In [23]:
# Only keep these columns:
df_oa_lsoa_region = df_oa_lsoa_region[[
    'LSOA11NM', 'LSOA11CD', 'RGN11NM', 'CTRY11NM'
]].drop_duplicates()

In [24]:
# Limit to England and Wales:
df_oa_lsoa_region = df_oa_lsoa_region[
    df_oa_lsoa_region['CTRY11NM'].isin(['England', 'Wales'])]

## Create new dataframe

Merge all of these DataFrames into one combined DataFrame with all of the useful information.

The only column name that is in common across all of these DataFrames is 'LSOA11CD'. Make sure there aren't any extra shared column names or the repeat columns will be given ugly suffixes.

In [25]:
dfs_to_merge = [
    df_lsoa_lhb,
    df_lsoa_icb_lad,
    df_lsoa_isdn,
    df_lsoa_scn,
    df_lsoa_ccg_stp,
    df_lsoa_ambo
]

In [26]:
df_combo = df_oa_lsoa_region.copy()
for df in dfs_to_merge:
    df_combo = pd.merge(df_combo, df, on='LSOA11CD', how='outer')

In [27]:
df_combo.head(3).T

,0,1,2
LSOA11NM,Hartlepool 006B,Hartlepool 005B,Hartlepool 006A
LSOA11CD,E01011966,E01011974,E01011965
RGN11NM,North East,North East,North East
CTRY11NM,England,England,England
LHB20CD,NaN,NaN,NaN
LHB20NM,NaN,NaN,NaN
LHB20NMW,NaN,NaN,NaN
LOC22CD,E38000247,E38000247,E38000247
LOC22NM,NHS North East and North Cumbria ICB - 16C,NHS North East and North Cumbria ICB - 16C,NHS North East and North Cumbria ICB - 16C
ICB22CD,E54000050,E54000050,E54000050


### Save a copy of the full data

In [28]:
df_combo.to_csv(os.path.join(paths.dir_tabular, paths.df_output_full), index=False)

### Save a copy with LSOA names

In [29]:
df_combo_with_lsoa = df_combo[[
    'LSOA11NM', 'LSOA11CD',
    'LOC22CD', 'LOC22NM',
    'ICB22CD', 'ICB22NM',
    'LHB20CD', 'LHB20NM', 'LHB20NMW',
]].copy()

In [30]:
df_combo_with_lsoa.to_csv(os.path.join(paths.dir_tabular, paths.df_output_with_lsoa), index=False)

### Save a copy without LSOA names

In [31]:
df_combo_without_lsoa = df_combo.drop(
    ['LSOA11NM', 'LSOA11CD'], axis='columns').copy()

# Limit the columns:
cols_to_drop = [
    'LAD22CD', 'LAD22NM',
    'CCG19CD', 'CCG19NM',
    'STP19CD', 'STP19NM',
    'SCN17CD', 'SCN17NM',
]
df_combo_without_lsoa = df_combo_without_lsoa.drop(
    cols_to_drop, axis='columns').drop_duplicates()

In [32]:
df_combo_without_lsoa.to_csv(os.path.join(paths.dir_tabular, paths.df_output_without_lsoa), index=False)